# RISC-V Business

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ramshreyas/Ethereum-Articles/blob/main/2/RISC-V_Business.ipynb)

!["RISC-V Business"](https://szmer.info/pictrs/image/6313c4e4-5b15-41ee-afa8-a7765cda4893.png)

Angelina Jolie in Hackers, 1995

It's almost 2025, and the great RISC revolution that has been looming for decades is finally threatening to materialize. The Ethereum Foundation is [funding a formal verification effort for RISC-V ZKVMs](https://verified-zkevm.org/), chip industry legends like Michael Keller are starting companies like [tenstorrent](https://tenstorrent.com/en/vision/tenstorrent-risc-v-and-chiplet-technology-selected-to-build-the-future-of-ai-in-japan), RISC-V International (the governing body for the RISC-V standard) has [ratified the RISC-V specifications](https://riscv.org/announcements/2024/04/risc-v-international-achieves-milestone-with-ratification-of-40-specifications-in-two-years/), laying the ground for an open standard that industry can now build upon, and Justin Drake in his ambitious [Beam Chain proposal](https://youtu.be/lRqnFrqpq4k) outlined how the onset of RISC-V can massively accelerate Ethereum's ponderous roadmap dramatically.

This article aims to introduce RISC-V, motivate its importance to the Ethereum vision, and, with a few toy code examples, connect the dots across the entire tech stack from high level code down to the metal.

---

### Why RISC? Faster ZK.

**ZK is Ethereum's future**

![The Verge](/assets/imgs/TheVerge.png)

As Vitalik outlines in his blog post about [The Verge](https://vitalik.eth.limo/general/2024/10/23/futures4.html) - the goal for this 

1. The zk Revolution in Ethereum
    Why zero-knowledge proofs (ZKPs) are critical for Ethereum’s scalability, privacy, and efficiency.
2. The Role of Hardware in Ethereum's Future
    High computational demands of zk-rollups, cryptography, and how specialized hardware can help.
3. Why RISC-V Matters
    An open, modular, and extensible ISA that aligns with Ethereum’s decentralized ethos.

---

### ZK Use Cases in Ethereum and How RISC-V Enables Them

1. zk-Rollups and Provers
    Challenges of zk-SNARK proving (e.g., field arithmetic, FFTs).
    How RISC-V extensions can optimize zk-rollup nodes.
2. zk-STARK Verifications
    The role of hashing and polynomial checks, and how RISC-V accelerates them.
3. Cryptographic Tooling
    Optimizing Ethereum cryptography (e.g., Keccak, BLS signatures) with RISC-V.

---

### RISC-V Features That Empower Ethereum Development

1. Open and Modular Design
    How RISC-V allows Ethereum teams to design custom hardware (e.g., ZKP accelerators).
2. Auditable and Trustless Hardware
    Aligning RISC-V’s transparency with Ethereum’s trustless ethos.
3. Portability Across Architectures
    Cross-platform compatibility for Ethereum clients and cryptographic tooling.

---

### Lightweight Code Examples: RISC-V in Action

1. A Simple ZKP Operation: Modular Arithmetic
    Code example: A RISC-V instruction for modular addition.
2. Optimizing Keccak on RISC-V
    Code snippet: A hashing operation in RISC-V.
3. FFT Steps in zk-SNARK Provers
    Code walkthrough: Using RISC-V to perform polynomial evaluations.

---

### How RISC-V Works: ISA Basics for Ethereum Developers

1. The ISA as the Software-Hardware Contract
    Explanation of ISAs and how they enable software portability.
2. Stack-Based Machines vs. Register-Based Architectures
    Contrasting the EVM with RISC-V for Ethereum developers.
3. RISC-V’s Modular Extensions
    How Ethereum-specific operations can be implemented efficiently.

---


### Why RISC-V is Uniquely Suited for Ethereum

1. Open Collaboration in Blockchain and Hardware
    Community-driven innovation in Ethereum and RISC-V.
2. Future-Proofing Ethereum with RISC-V
    RISC-V’s adaptability for evolving zk and cryptographic needs.
3. A Decentralized Approach to Hardware Design
    Enabling hardware decentralization with open-source RISC-V.

---

### Conclusion

1. Call to Action for Ethereum Developers
    Start experimenting with RISC-V and contribute to its ecosystem.
2. The Path Forward for RISC-V and Ethereum
    Collaboration opportunities and the importance of open hardware for the future of decentralized technologies.

---